Zadanie polegało na przeprowadzeniu zbiorowego eksperymentu z wykorzystaniem klasyfikatorów liniowych. Dla każdego z nich należało przetestować te same dane po podziale na zadany rozmiar części testowej.
Wyniki z dla poszczególnych zbiorów danych zostały przedstawione w strukturze `DataFrame` oraz `pivot_table`.

Importy:

In [1]:
import time
from typing import Tuple
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.datasets import fetch_rcv1, fetch_openml
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from scipy.sparse import csr_matrix, csc_matrix
import warnings

Klasa służąca do wykonywania testów związanych z liniowymi klasyfikatorami.

Opis parametrów:
* `max_iter` - maksymalna liczba iteracji dla wykorzystanych klasyfikatorów
* `test_sizes` - tablica zawierająca rozmiary części uczącej po podziale danych
* `regularization_params` - tablica zawierająca listę parametrów regularyzacyjnych do przetestowania - wykorzystywana w `GridSearchCV`
* `results_count_per_test_size` - liczba iteracji do wykonania na każdy z rozmiarów testowych. Wykorzystywane do wielokrotnego podziału tych samych danych, a następnie uśredniania zebranych wyników
* `classifiers` - klasyfikatory wykorzystane w eksperymencie:
    * `SVC` jako wersja liniowa
    * `MLPClassifier` jako wersja liniowa bez warstw ukrytych
    * `LogisticRegression` z funkcjami kary `L1` oraz `L2`
    
Własna implementacja `SVM2` nie została wykorzystana ze względu na niezidentyfikowane problemy z działaniem biblioteki `cvxopt`.

Opis metod:
* `experiment` - główna metoda wykonująca eksperyment na wszytkich zbiorach danych, zbierająca wyniki z każdego zbioru danych w jedną tabelę
* `experiment_for_dataset` - metoda odpowiedzialna za wykonanie eksperymentu dla przekazanych danych. Dla każdego rozmiaru testowego dzieli wykonuje zadaną liczbę iteracji, dzieląc te same dane, a następnie po zadanej iteracji uśrednia zebrane wyniki dla danego rozmiaru zbioru testowego. Wraz z parametrem `verbose` pozwala śledzić na ekranie postęp obliczeń
* `clf_experiment` - metoda, która dla przekazanego klasyfikatora oraz danych wykonuje uczenie mierząc jego czas, oraz wyznacza miary `accuracy`, `f1` oraz `auc` zarówno dla zbioru uczącego jak i testowego. Zbiera wszystkie wyniki w całość i zwraca `pd.Series`
* `calc_classification_quality` - metoda wykorzystywana przez powyższą do wyznaczania miar
* `get_sonar_data` - zwraca zbiór `sonar_csv` jako atrybuty oraz klasy
* `get_reuters_data` - zwraca zbiór Reuters: atrybuty jako macierz rzadka oraz klasy jako tablica `numpy`, ponieważ zastosowane klasyfikatory wymagają decyzji jako danych gęstych
* `get_mnist_data` - zwraca zbiór mnist: atrybuty jako macierz rzadka oraz wybrana cyfra jako klasa pozytywna, a reszta jako negatywna

In [2]:
class LinearClassifierTest:
    def __init__(self, test_sizes: np.array = np.logspace(-1, -0.5, num=2), regularization_params: list = None, results_count_per_test_size: int = 3):
        self.max_iter = 1000
        self.test_sizes = test_sizes
        self.regularization_params = regularization_params if not None else [10 ** i for i in range(-3, 2)]
        self.results_count_per_test_size = results_count_per_test_size

        penalties = [
            'l1',
            'l2',
            # 'elasticnet'  # only solver saga can handle with it, but it throws error on calc
        ]
        self.classifiers = {
            'svc': SVC(kernel='linear', max_iter=self.max_iter),
            # 'svm2': Svm2(),
            'mlp': MLPClassifier(hidden_layer_sizes=(), max_iter=self.max_iter),
            **{f'logistic_regression_{penalty}': LogisticRegression(penalty=penalty, max_iter=self.max_iter, solver='saga') for penalty in penalties}
        }

        # uncomment to run with GridSearch. Warning!! Calculating time will drastically increase, especially for sparse data.
        # self.classifiers = {
        #     'svc': GridSearchCV(SVC(kernel='linear', max_iter=self.max_iter), param_grid={'C': self.regularization_params}),
        #     # 'svm2': GridSearchCV(Svm2(), param_grid={'c': self.regularization_params}),
        #     'mlp': GridSearchCV(MLPClassifier(hidden_layer_sizes=(), max_iter=self.max_iter), param_grid={'alpha': self.regularization_params}),
        #     **{f'logistic_regression_{penalty}': GridSearchCV(LogisticRegression(penalty=penalty, max_iter=self.max_iter, solver='saga'), param_grid={'C': self.regularization_params}) for penalty in penalties}
        # }

        self.data_table = None

    def experiment(self, verbose: bool = False):
        dataset_method_generators = {
            'sonar': self.get_sonar_data,
            'reuters': self.get_reuters_data,
            'mnist': self.get_mnist_data,
        }

        for dataset_name, method in dataset_method_generators.items():
            if verbose:
                print(f'\n--------------- {dataset_name} ---------------')

            x, y = method()

            results = self.experiment_for_dataset(x, y, verbose)
            results['dataset'] = dataset_name

            self.data_table = results if self.data_table is None else pd.concat([self.data_table, results], ignore_index=True)

    def experiment_for_dataset(self, x: np.array, y: np.array, verbose: bool = False, verbose_level: int = 2) -> pd.DataFrame:
        mean_results_count = len(self.classifiers.keys()) * self.test_sizes.size
        mean_results_storage = [0 for _ in range(mean_results_count)]
        mean_results_iter = 0

        for test_size_iter, test_size in enumerate(self.test_sizes):
            results_storage = {clf_name: [0] * self.results_count_per_test_size for clf_name in self.classifiers.keys()}
            t1_test_size = time.time()

            for results_iter in range(self.results_count_per_test_size):
                separated_data = train_test_split(x, y, test_size=test_size)
                t1_inner_loop = time.time()

                for clf_iter, (clf_name, clf) in enumerate(self.classifiers.items()):
                    t1_clf_loop = time.time()
                    experiment_results = self.clf_experiment(clf, separated_data)
                    experiment_results['clf'] = clf_name

                    results_storage[clf_name][results_iter] = experiment_results
                    
                    if verbose and verbose_level > 2:
                        t2_clf_loop = time.time()
                        time_clf_loop = t2_clf_loop - t1_clf_loop
                        progres_percent = (clf_iter + 1) / len(self.classifiers.keys()) * 100

                if verbose and verbose_level > 1:
                    t2_inner_loop = time.time()
                    time_inner_loop = t2_inner_loop - t1_inner_loop
                    progress_percent = (results_iter + 1) / self.results_count_per_test_size * 100

                    print(f'\t\tProgress inner loop: {progress_percent:.2f}% - {time_inner_loop:.4f}s')

            for results_iter, (clf_name, results_for_clf) in enumerate(results_storage.items()):
                results = pd.DataFrame(results_for_clf)
                mean_results = results.mean()

                mean_results['clf'] = clf_name
                mean_results['test_size'] = test_size
                mean_results_storage[mean_results_iter] = mean_results

                mean_results_iter += 1

            if verbose:
                t2_test_size = time.time()
                time_test_size = t2_test_size - t1_test_size
                progress_percent = (test_size_iter + 1) / self.test_sizes.size * 100

                print(f'\tProgress test size: {progress_percent:.2f}% - {time_test_size:.4f}s')

        return pd.DataFrame(mean_results_storage)

    def clf_experiment(self, clf, separated_data: tuple) -> pd.Series:
        x_train, x_test, y_train, y_test = separated_data

        t1_fit = time.time()
        clf.fit(x_train, y_train)
        t2_fit = time.time()
        time_fit = t2_fit - t1_fit

        classification_quality_train = self.calc_classification_quality(clf, x_train, y_train, 'train')
        classification_quality_test = self.calc_classification_quality(clf, x_test, y_test, 'test')

        return pd.Series({
            'fit_time': time_fit,
            **classification_quality_train,
            **classification_quality_test
        })

    @staticmethod
    def calc_classification_quality(clf, x: np.array, y: np.array, data_type_label: str = 'train'):
        t1_predict = time.time()
        y_predicted = clf.predict(x)
        t2_predict = time.time()
        time_predict = t2_predict - t1_predict

        accuracy = metrics.accuracy_score(y, y_predicted)
        f1_score = metrics.f1_score(y, y_predicted)
        auc_score = metrics.roc_auc_score(y, y_predicted)

        return {
            f'predict_time_{data_type_label}': time_predict,
            f'accuracy_{data_type_label}': accuracy,
            f'f1_{data_type_label}': f1_score,
            f'auc_{data_type_label}': auc_score,
        }

    def get_sonar_data(self) -> Tuple:
        sonar_data = pd.read_csv('sonar_csv.csv')
        y = self.normalize_decisions(sonar_data[sonar_data.columns[-1]])
        x = sonar_data.drop(sonar_data.columns[-1], axis=1).to_numpy()

        return x, y

    @staticmethod
    def get_reuters_data() -> Tuple:
        rcv1 = fetch_rcv1()
        x = rcv1['data'] > 0
        xr = x[:, 2]
        y = rcv1['target'][:, 5]

        # return xr, y.toarray().ravel()
        return x, y.toarray().ravel()

    @staticmethod
    def get_mnist_data(class_digit: int = 5) -> Tuple:
        x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")

        # return csc_matrix(x)[:, 400], y == str(class_digit)  # 407 column for minimalize sparse
        return csc_matrix(x), y == str(class_digit)

    @staticmethod
    def normalize_decisions(d) -> np.array:
        d_normalized = np.ones(d.size).astype("int8")
        d_normalized[d == np.unique(d)[0]] = -1

        return d_normalized

Funkcje pomocnicze do wykonania oraz przedstawienia wyników eksperymentu:

In [3]:
def display_linear_classifiers_experiment_results(results: pd.DataFrame):
    print('\nWhole table from experiment:')
    print(results.to_markdown())

    columns = ['fit_time', 'predict_time_train', 'accuracy_train', 'f1_train', 'auc_train', 'predict_time_test', 'accuracy_test', 'f1_test', 'auc_test', 'test_size']
    print('\nTable with aggregation by: mean, min, max')
    print(results.agg({column: ['mean', 'min', 'max'] for column in columns}).to_markdown())

    pivot_table = pd.pivot_table(results, values=['accuracy_test', 'f1_test', 'auc_test'], index=['clf', 'test_size'], aggfunc=np.mean)
    print('\nPivot table by classifier and test size')
    print(pivot_table)


def linear_classifiers_experiment():
    linear_classifier_test = LinearClassifierTest()

    print('\n--------------- sonar ---------------')
    linear_classifier_test.test_sizes = np.logspace(-1, -0.5, num=30)
    linear_classifier_test.results_count_per_test_size = 100

    results = linear_classifier_test.experiment_for_dataset(*linear_classifier_test.get_sonar_data(), verbose=True, verbose_level=1)
    display_linear_classifiers_experiment_results(results)

    print('\n\n--------------- mnist ---------------')
    linear_classifier_test.test_sizes = np.logspace(-1, -0.5, num=5)
    linear_classifier_test.results_count_per_test_size = 5

    results = linear_classifier_test.experiment_for_dataset(*LinearClassifierTest.get_mnist_data(), verbose=True)
    display_linear_classifiers_experiment_results(results)

    print('\n\n--------------- reuters ---------------')
    linear_classifier_test.test_sizes = np.array([np.logspace(-1, -0.5, num=50)[-20]])
    linear_classifier_test.results_count_per_test_size = 1

    results = linear_classifier_test.experiment_for_dataset(*LinearClassifierTest.get_reuters_data(), verbose=True, verbose_level=3)
    display_linear_classifiers_experiment_results(results)

Wykonanie eksperymentu:

In [4]:
if __name__ == '__main__':
    with warnings.catch_warnings() and pd.option_context('display.max_rows', None, 'display.max_columns', None):
        warnings.simplefilter("ignore")

        linear_classifiers_experiment()


--------------- sonar ---------------
	Progress test size: 3.33% - 38.0142s
	Progress test size: 6.67% - 38.4673s
	Progress test size: 10.00% - 37.7358s
	Progress test size: 13.33% - 37.7561s
	Progress test size: 16.67% - 38.0275s
	Progress test size: 20.00% - 37.6024s
	Progress test size: 23.33% - 37.5727s
	Progress test size: 26.67% - 37.3923s
	Progress test size: 30.00% - 37.2516s
	Progress test size: 33.33% - 37.4699s
	Progress test size: 36.67% - 37.3143s
	Progress test size: 40.00% - 37.3763s
	Progress test size: 43.33% - 37.3005s
	Progress test size: 46.67% - 37.4057s
	Progress test size: 50.00% - 37.2849s
	Progress test size: 53.33% - 37.1891s
	Progress test size: 56.67% - 37.2674s
	Progress test size: 60.00% - 37.0793s
	Progress test size: 63.33% - 37.2515s
	Progress test size: 66.67% - 37.2044s
	Progress test size: 70.00% - 37.1108s
	Progress test size: 73.33% - 37.3167s
	Progress test size: 76.67% - 36.9360s
	Progress test size: 80.00% - 36.7224s
	Progress test size: 83.33%

Wnioski:
* Dla zbioru 'sonar' najwolniejszy w uczeniu był `MLPClassifier` w każdym przypadku, natomiast dla danych rzadkich był on zdecydowanie najszybszy - znacznie szybszy od pozostałych klasyfikatorów.
* Dla danych rzadkich najwolniejsza była `LogisticRegression` wraz funkcją kary `L1`. Gigantyczny czas uczenia w przypadku zbioru danych 'Reuters'. Wcześniejsza wiedza o tym pozwoliła by wykluczyć ten klasyfikator z testów i dokonać eksperyment na większej ilości parametrów.
* Porównując dane otrzymane dla zbiorów 'sonar' oraz 'mnist' wynika, że im więcej danych znajdzie się w części uczącej, tym lepsze dokładności dla części testowej zostaną uzyskane.
* `SVC` miał najdłuższe czasy predykcji niezależnie od zbioru danych. Im większy zbiór, tym większy czas predykcji.
* Powyższy klasyfikator uzyskał najgorsze wyniki (auccuracy_score, f1_score, roc_auc_score) w porównaniu do pozostałych dla zbiorów danych 'Reuters' oraz 'mnist'. Wyniki te znacznie odstają od pozostałych. 
* Brak zastosowania własnej implementacji `SVM2` wynika z niezidentyfikowanych błędów związanych z biblioteką `cvxopt`. Natomiast brak `LogisticRegression` z funkcją kary `elasticnet` wynika z niedokładnego przeczytania dokumentacji oraz niezastosowaniu parametru `l1_ratio=0.5`, którego brak powodował błędy w działaniu programu.
* Podsumowanie zbiorów:
    * 'sonar'
        * `MLPClassifier` najwolniejszy czas uczenia, 
        * `SVC` najlepsze wyniki zarówno na części uczącej jak i testowej
    * 'mnist' - 
        * `LogisticRegression` z funkcją kary `L1` - najwolniejszy czas uczenia, 
        * `SVC` najdłuższy czas predykcji i odstające dokładności klasyfikacji, 
        * `MLPClassifier` najszybszy jeżeli chodzi o czas uczenia, 
        * `LogisticRegression` niezależnie od funkcji kary daje najlepsze dokładności klasyfikacji, zarówno na części uczącej jak i na cześci testowej
    * 'Reuters' - 
        * `LogisticRegression` z funkcją kary `L1` - gigantyczny czas uczenia,
        * `SVC` najdłuższy czas predykcji i odstające dokładności klasyfikacji,
        * `MLPClassifier` najszybszy jeżeli chodzi o czas uczenia,
        * `SVC` jako jedyny odstaje dokładnościami klasyfikacji od konkurencyjnych klasyfikatorów, które dają porównywalne, bliskie siebie wartości jako dokładności klasyfikacji